In [34]:
import pandas as pd
import numpy as np
import pymongo
import json
import os
import pickle
import sys

In [16]:
# import the connecting credentials 
sys.path.append('../')
from secret import credentials
sys.path.remove('../')

In [17]:
## mongo API connection
sec = credentials()
mongo_uri = sec.connect_string

myclient = pymongo.MongoClient(mongo_uri)

In [18]:
print(myclient.list_database_names())

['bookdb', 'admin', 'local']


In [20]:
mydb = myclient['bookdb']
col_users = mydb['users']

In [21]:
# Query the data (this example retrieves all documents in the collection)
data = list(col_users.find({}))
# Convert the data to a Pandas DataFrame
users = pd.DataFrame(data)

In [22]:
users.drop('_id', axis=1, inplace= True)

In [24]:
users.rename(columns={
    'User-ID':'user_id'
}, inplace=True)

In [25]:
users.head()

,user_id,Location,country,city,Age
0,1.0,"nyc, new york, usa",usa,nyc,NaN
1,2.0,"stockton, california, usa",usa,stockton,18.0
2,3.0,"moscow, yukon territory, russia",russia,moscow,NaN
3,4.0,"porto, v.n.gaia, portugal",portugal,porto,17.0
4,5.0,"farnborough, hants, united kingdom",united kingdom,farnborough,NaN


In [26]:
ratings = pd.read_csv('../py_objects/final_rating.csv', index_col=0).reset_index().drop(columns=['index'])
ratings

,title,num_of_rating,user_id,ISBN,rating,author,year,Publisher,Image_URL
0,10 Lb. Penalty,35,39281,0515123471,0,Dick Francis,1998,Jove Books,http://images.amazon.com/images/P/0515123471.0...
1,10 Lb. Penalty,35,49109,0515123471,5,Dick Francis,1998,Jove Books,http://images.amazon.com/images/P/0515123471.0...
2,10 Lb. Penalty,35,69405,0515123471,0,Dick Francis,1998,Jove Books,http://images.amazon.com/images/P/0515123471.0...
3,10 Lb. Penalty,35,76352,0515123471,0,Dick Francis,1998,Jove Books,http://images.amazon.com/images/P/0515123471.0...
4,10 Lb. Penalty,35,83287,0515123471,10,Dick Francis,1998,Jove Books,http://images.amazon.com/images/P/0515123471.0...
...,...,...,...,...,...,...,...,...,...
93853,"\O\"" Is for Outlaw""",105,234359,0805059555,0,Sue Grafton,1999,Henry Holt &amp; Company,http://images.amazon.com/images/P/0805059555.0...
93854,"\O\"" Is for Outlaw""",105,234623,0805059555,0,Sue Grafton,1999,Henry Holt &amp; Company,http://images.amazon.com/images/P/0805059555.0...
93855,"\O\"" Is for Outlaw""",105,236283,0805059555,10,Sue Grafton,1999,Henry Holt &amp; Company,http://images.amazon.com/images/P/0805059555.0...
93856,"\O\"" Is for Outlaw""",105,238120,0805059555,0,Sue Grafton,1999,Henry Holt &amp; Company,http://images.amazon.com/images/P/0805059555.0...


In [27]:
user_merge_ratings = users.merge(ratings, on = "user_id")
user_merge_ratings.head()

,user_id,Location,country,city,Age,title,num_of_rating,ISBN,rating,author,year,Publisher,Image_URL
0,254.0,"minneapolis, minnesota, usa",usa,minneapolis,24.0,1984,76,0451524934,9,George Orwell,1990,Signet Book,http://images.amazon.com/images/P/0451524934.0...
1,254.0,"minneapolis, minnesota, usa",usa,minneapolis,24.0,A Bend in the Road,115,0446527785,0,Nicholas Sparks,2001,Warner Books,http://images.amazon.com/images/P/0446527785.0...
2,254.0,"minneapolis, minnesota, usa",usa,minneapolis,24.0,ANGELA'S ASHES,81,0684874350,0,Frank McCourt,1996,Scribner,http://images.amazon.com/images/P/0684874350.0...
3,254.0,"minneapolis, minnesota, usa",usa,minneapolis,24.0,American Gods,94,0380789035,10,Neil Gaiman,2002,HarperTorch,http://images.amazon.com/images/P/0380789035.0...
4,254.0,"minneapolis, minnesota, usa",usa,minneapolis,24.0,Angela's Ashes (MMP) : A Memoir,92,0684872153,0,Frank McCourt,1999,Scribner,http://images.amazon.com/images/P/0684872153.0...


In [73]:
def clean_us(country):
    if country == 'us':
        return 'usa'
    else:
        return country

In [74]:
user_merge_ratings['country'] = user_merge_ratings['country'].apply(clean_us)

In [75]:
user_merge_ratings.head()

,user_id,Location,country,city,Age,title,num_of_rating,ISBN,rating,author,year,Publisher,Image_URL
0,254.0,"minneapolis, minnesota, usa",usa,minneapolis,24.0,1984,76,0451524934,9,George Orwell,1990,Signet Book,http://images.amazon.com/images/P/0451524934.0...
1,254.0,"minneapolis, minnesota, usa",usa,minneapolis,24.0,A Bend in the Road,115,0446527785,0,Nicholas Sparks,2001,Warner Books,http://images.amazon.com/images/P/0446527785.0...
2,254.0,"minneapolis, minnesota, usa",usa,minneapolis,24.0,ANGELA'S ASHES,81,0684874350,0,Frank McCourt,1996,Scribner,http://images.amazon.com/images/P/0684874350.0...
3,254.0,"minneapolis, minnesota, usa",usa,minneapolis,24.0,American Gods,94,0380789035,10,Neil Gaiman,2002,HarperTorch,http://images.amazon.com/images/P/0380789035.0...
4,254.0,"minneapolis, minnesota, usa",usa,minneapolis,24.0,Angela's Ashes (MMP) : A Memoir,92,0684872153,0,Frank McCourt,1999,Scribner,http://images.amazon.com/images/P/0684872153.0...


In [6]:
# user_merge_ratings = user_merge_ratings[user_merge_ratings['num_of_rating'] >= 50]

In [77]:
user_merge_ratings.groupby(['country'])['num_of_rating'].sum().sort_values(ascending=False)

country
usa                   6083221
canada                 606807
united kingdom         153930
australia               82995
n/a                     81756
portugal                59694
                        50467
malaysia                34199
new zealand             33742
germany                 27163
spain                   24716
netherlands             19954
iran                    13122
france                  11476
italy                    9191
china                    8848
romania                  8101
brazil                   7609
belgium                  4936
switzerland              4811
qatar                    4328
dominican republic       3982
finland                  2100
austria                  2092
japan                    1951
sweden                   1555
denmark                   448
Name: num_of_rating, dtype: int64

In [78]:
region_unique = user_merge_ratings['country'].unique().tolist()

In [79]:
region_unique.remove('')
region_unique.remove('n/a')

In [81]:
region_unique

['usa',
 'united kingdom',
 'canada',
 'spain',
 'romania',
 'portugal',
 'new zealand',
 'australia',
 'malaysia',
 'netherlands',
 'germany',
 'switzerland',
 'france',
 'italy',
 'belgium',
 'dominican republic',
 'austria',
 'china',
 'qatar',
 'finland',
 'japan',
 'sweden',
 'iran',
 'denmark',
 'brazil']

In [82]:
with open('../py_objects/region_unique.pkl', 'wb') as file:
    pickle.dump(region_unique, file)    

In [84]:
# store the saved dataframe
user_merge_ratings.to_csv('../data/final_merge_table.csv', index=False)